In [1]:
from scipy.io import loadmat
import numpy as np
import pandas as pd
from PIL import Image
import glob
from utils import *
from tqdm import tqdm
from sklearn.model_selection import train_test_split
import imp

Using TensorFlow backend.


In [3]:
import utils
imp.reload(utils)
from utils import *

If there is no dataset and utils:

In [ ]:
!wget https://raw.githubusercontent.com/rostyslavb/GazeEstimator/master/model/utils.py
!wget http://datasets.d2.mpi-inf.mpg.de/MPIIGaze/MPIIGaze.tar.gz
!tar -xzf MPIIGaze.tar.gz

## Prepare data

### Gather Data from Structure

In [2]:
index, image, pose, gaze = gather_all_data('./MPIIGaze/Data/Normalized')

100%|██████████| 521/521 [00:14<00:00, 36.22it/s]


In [3]:
image = image / 255
gaze = gaze3Dto2D(gaze)
pose = pose3Dto2D(pose)

In [4]:
def print_shapes(titles, items):
    for title, item in zip(titles, items):
        print((title+':').ljust(15) + str(item.shape))

print_shapes(['Indices', 'Images', 'Poses', 'Gazes'], (index, image, pose, gaze))

Indices:       (427316, 4)
Images:        (427316, 36, 60, 1)
Poses:         (427316, 2)
Gazes:         (427316, 2)


### Train/test split

In [5]:
random_state = 42
index_train, index_test = train_test_split(pd.DataFrame(index),
                                            stratify=index[:, [0, -1]],
                                            test_size=0.2,
                                            random_state=random_state)

index_train = index_train.index
index_test = index_test.index

**Train:**

In [6]:
print_shapes(['Indices',], (index_train,))

Indices:       (341852,)


**Test:**

In [7]:
print_shapes(['Indices',], (index_test,))

Indices:       (85464,)


## Create NN

In [8]:
from keras.layers import Input, Conv2D, MaxPool2D, Dense, Concatenate, Flatten, Dropout
from keras.initializers import RandomNormal, glorot_normal, glorot_uniform
from keras.models import Model
from keras import backend as K
from keras.callbacks import TensorBoard
from keras.optimizers import Adam
from keras.models import Model

import tensorflow as tf

Using TensorFlow backend.


### Model

In [25]:
with tf.Graph().as_default():
    
    ### LAYERS ###

    # input
    input_img = Input(shape=(36, 60, 1), name='InputNormalizedImage')
    input_pose = Input(shape=(2,), name='InputHeadPose')

    # convolutional
    conv1 = Conv2D(filters=20,
                   kernel_size=(5, 5),
                   strides=(1, 1),
                   kernel_initializer=RandomNormal(mean=0.0, stddev=0.01, seed=42),
                   bias_initializer='zeros',
                   name='conv1'
                  )(input_img)
    pool1 = MaxPool2D(pool_size=(2, 2),
                      strides=(2, 2),
                      padding='valid',
                      name='maxpool1'
                     )(conv1)
    conv2 = Conv2D(filters=50,
                   kernel_size=(5, 5),
                   strides=(1, 1),
                   kernel_initializer=RandomNormal(mean=0.0, stddev=0.001, seed=42),
                   bias_initializer='zeros',
                   name='conv2'
                  )(pool1)
    pool2 = MaxPool2D(pool_size=(2, 2),
                      strides=(2, 2),
                      padding='valid',
                      name='maxpool2'
                     )(conv2)

    flatt = Flatten(name='flatt')(pool2)

    # inner product 1
    dense1 = Dense(units=500,
                  activation='relu',
                  kernel_initializer=glorot_normal(seed=42),
                  bias_initializer='zeros',
                  name='ip1'
                 )(flatt)

    dropout = Dropout(rate=0.1)(dense1)

    # concatanate with head pose
    cat = Concatenate(axis=-1, name='concat')([dropout, input_pose])

    # inner product 2
    dense2 = Dense(units=2,
                  kernel_initializer=glorot_normal(seed=42),
                  bias_initializer='zeros',
                  name='ip2'
                 )(cat)
    
    ### OPTIMIZER ###
    adam = Adam(lr=1e-3)
    
    ### CALLBACKS ###
    tbCallBack = TensorBoard(log_dir='./log',
                         histogram_freq=0,
                         write_graph=True,
                         write_images=True)
    
    ### COMPILE MODEL ###
    model = Model([input_img, input_pose], dense2)
    model.compile(optimizer=adam, loss='mse', metrics=[angle_accuracy])

### Train

In [ ]:
with tf.Graph().as_default():
    model.fit(x=[image[index_train], pose[index_train]], y=gaze[index_train],
              batch_size=500,
              verbose=1,
              epochs=1000,
              validation_data=([image[index_test], pose[index_test]], gaze[index_test]),
              callbacks=[tbCallBack])